# くずし字データの前処理実験

このノートブックでは、くずし字データの前処理（列検出）の結果を可視化して確認します。

In [1]:
import cv2
import yaml
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

import sys

sys.path.append("../")

from scripts.data_preprocessing import CharacterBox, detect_text_columns

In [2]:
def visualize_columns(image: np.ndarray, columns: list[list[CharacterBox]], figsize: tuple[int, int] = (15, 15)) -> None:
    """列検出結果を可視化

    Args:
        image (np.ndarray): 入力画像
        columns (list[list[CharacterBox]]): 検出された列のリスト
        figsize (tuple[int, int], optional): 図のサイズ. Defaults to (15, 15).
    """
    plt.figure(figsize=figsize)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # 列ごとに異なる色を使用
    colors = plt.cm.rainbow(np.linspace(0, 1, len(columns)))

    for i, column in enumerate(columns):
        # 列の領域を計算
        x1 = min(char.x1 for char in column)
        y1 = min(char.y1 for char in column)
        x2 = max(char.x2 for char in column)
        y2 = max(char.y2 for char in column)

        # 列の領域を描画
        plt.plot([x1, x2, x2, x1, x1], [y1, y1, y2, y2, y1], color=colors[i], linewidth=2, label=f"Column {i + 1}")

        # 文字のバウンディングボックスを描画
        for char in column:
            plt.plot(
                [char.x1, char.x2, char.x2, char.x1, char.x1],
                [char.y1, char.y1, char.y2, char.y2, char.y1],
                color=colors[i],
                linewidth=1,
                alpha=0.5,
            )

    plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.axis("off")
    plt.tight_layout()
    plt.show()

## パラメータの設定

列検出のパラメータを設定します。

In [9]:
# DBSCANのパラメータ
params = {
    "eps_ratio": 0.2,  # 文字の幅の中央値に対するepsの割合
    "min_samples": 1,  # クラスタを形成するための最小サンプル数
}

## データの読み込みと前処理

画像とアノテーションを読み込み、列検出を実行します。

In [ ]:
# 設定の読み込み
with open("../config/preprocessing.yaml", "r") as f:
    config = yaml.safe_load(f)

# データディレクトリの設定
data_dir = Path(config["data_dir"])
raw_dir = ".." / data_dir / "raw" / "dataset"

# サンプル画像の処理
page_dirs = list(raw_dir.iterdir())
print(len(page_dirs))
for page_dir in page_dirs[21:22]:  # 最初の3ページのみ処理
    if not page_dir.is_dir():
        continue

    # 画像ディレクトリとcoordinate.csvの確認
    images_dir = page_dir / "images"
    coord_file = page_dir / f"{page_dir.name}_coordinate.csv"

    if not images_dir.exists() or not coord_file.exists():
        continue

    # 各画像の処理
    count = 0
    for image_path in images_dir.glob("*.jpg"):
        print(f"\nProcessing {image_path.name}")

        # 画像の読み込み
        image = cv2.imread(str(image_path))

        # 文字座標の読み込み
        df = pd.read_csv(coord_file)

        # 画像名が一致する行のみ抽出
        df = df[df["Image"] == image_path.name.split("/")[-1].split(".")[0]]

        # 文字のバウンディングボックスを作成
        char_boxes = []
        for _, row in df.iterrows():
            char_boxes.append(
                CharacterBox(
                    x1=row["X"],
                    y1=row["Y"],
                    x2=row["X"] + row["Width"],
                    y2=row["Y"] + row["Height"],
                    unicode_id=row["Unicode"],
                )
            )

        # 列の検出
        text_columns = detect_text_columns(char_boxes, eps_ratio=params["eps_ratio"], min_samples=params["min_samples"])

        print(f"Detected {len(text_columns)} columns")

        # 結果の可視化
        visualize_columns(image, text_columns)

        count += 1
        if count >= 3:
            break

## パラメータの調整実験

異なるパラメータで結果を比較します。

In [ ]:
# サンプル画像を1枚選択
sample_image_path = list(page_dirs[0].glob("images/*.jpg"))[0]
sample_coord_file = page_dirs[0] / f"{page_dirs[0].name}_coordinate.csv"

# 画像の読み込み
image = cv2.imread(str(sample_image_path))

# 文字座標の読み込み
df = pd.read_csv(sample_coord_file)
char_boxes = []
for _, row in df.iterrows():
    char_boxes.append(
        CharacterBox(
            x1=row["X"], y1=row["Y"], x2=row["X"] + row["Width"], y2=row["Y"] + row["Height"], unicode_id=row["Unicode"]
        )
    )

# 異なるパラメータで実験
eps_ratios = [0.05, 0.1, 0.2]
min_samples_list = [1, 2, 3]

for eps_ratio in eps_ratios:
    for min_samples in min_samples_list:
        print(f"\neps_ratio: {eps_ratio}, min_samples: {min_samples}")

        # 列の検出
        text_columns = detect_text_columns(char_boxes, eps_ratio=eps_ratio, min_samples=min_samples)

        print(f"Detected {len(text_columns)} columns")

        # 結果の可視化
        visualize_columns(image, text_columns)